## widget notebooks: plumbing

building a nice widget demo often takes a bit of fiddling

In [ ]:
from __future__ import annotations

import sys

> 💡 `%pip` doesn't work when importing a notebook in lite

In [ ]:
if __name__ == "__main__" and "pyodide" in sys.modules:
    %pip install ipywidgets

In [ ]:
from typing import Any

from ipywidgets import HTML, FloatSlider, HBox, VBox

we build some free-form HTML.

In [ ]:
body = HTML("nothing to see yet", layout={"width": "95%"})
body

In [ ]:
sliders = {
    x: FloatSlider(description=x, min=-10, max=10, orientation="vertical", layout={"flex": "1"})
    for x in "abcdef"
}

In [ ]:
slide_box = HBox([*sliders.values()], layout={"width": "100%", "padding": "0 0 0 20%"})
app = VBox([body, slide_box])
app

then wire up some interactivity

In [ ]:
def on_slide(*changes: Any) -> None:
    wid = app.comm.comm_id
    lines = [
        f"""
        <style>
            .inb-demo-{wid} {{
                border-spacing: 0.5em;
                td, th {{ text-align: right; width: 10%; }}
                td[title*='is -'] {{ color: var(--jp-warn-color0); }}
                td[title*='is 0.0'] {{ opacity: 0.5; }}
            }}</style>
        <h2>Multiplication</h2>
        <table class="inb-demo-{wid}" width="100%">
        <tr><th>&times;</th>""",
        *[f"<th>{x}</th>" for x in sliders],
        "</tr>",
    ]
    for x, xs in sliders.items():
        xv = xs.value
        lines += [f"<tr><th>{x}</th>"]
        for y, ys in sliders.items():
            yv = ys.value
            xy = round(xv * yv, 1)
            xy = xy or 0.0
            lines += [
                f"""<td title="{x} times {y} is {xy} ">
                {xy}
                </td>"""
            ]
        lines += ["</tr>"]
    body.value = "\n".join([*lines, "</table>"])


[s.observe(on_slide, "value") for s in sliders.values()]
on_slide()

for restart and run all, it's nice to show the app again.

In [ ]:
app